# Load Data From Tushare
We load data from 2018-2021. Due to some constraint of platform, we download thses data year by year and save seperately. After that we process these data.

In [ ]:
import tushare as ts
import pandas as pd
import numpy as np

print(ts.__version__)

In [ ]:
# register token
token = '' # your token
ts.set_token(token)
pro = ts.pro_api()

In [ ]:
# got calendar from date range
start_date = '20180101'
end_date = '20210101'
calendar = pro.trade_cal(exchange='SSE', is_open='1', 
                            start_date=start_date, 
                            end_date=end_date, 
                            fields='cal_date')
# check all stocks exist in market today
stocks = pro.query('stock_basic', exchange='', list_status='L', market = '主板') # 主板/创业板/科创板/CDR/北交所
ts_code_list = ','.join(stocks.ts_code.values)
print(calendar.shape, stocks.shape)
calendar.tail()

In [ ]:
# get fundamental data
# pick up market cap [5,30] billion
base_universe = pro.bak_daily(trade_date='20180102', 
                   fields='trade_date, ts_code, name, float_mv, total_mv, pe, turn_over, industry')
filte_stock = base_universe.loc[(base_universe.total_mv>=50) & (base_universe.total_mv<=300)]
filte_stock

In [ ]:
from helper import download_helper

In [ ]:
# load total stock daily date for one year
ts_code_list = filte_stock.ts_code.values
all_stocks = download_helper.get_Daily_All(ts, ts_code_list, start_date, end_date)
print(all_stocks.shape)
all_stocks

In [ ]:
# save data
universe = all_stocks.drop_duplicates()
universe.to_csv(start_date +'-'+ end_date + '.csv')
filte_stock.drop_duplicates().to_csv('fundamental_' + start_date +'.csv')

# Load Data by File
if we load data from saved file. 

In [ ]:
# load data from csv
universe = pd.read_csv('20180101-20210101.csv').iloc[:,1:]
fundamental = pd.read_csv('fundamental_20180101.csv').iloc[:,1:]

# Process Data
1. filter ma_v_120 top 500 stocks
2. add 'date' column as datetime type, and deascanding time
3. add industry infomation and boll indicator to stock 

In [ ]:
from helper.factor_helper import IndicatorHelper, AverageByWindow
# pick average amount 120 days top 500
universe = AverageByWindow(universe).top(500, index='trade_date', ticker_column='ts_code', value_column='ma_v_120')

# add bollinger as indicator which will be used as a custom factor later
ind_helper = IndicatorHelper(universe)
tech_indicator_list = ['boll_ub','boll_lb']
universe = ind_helper.add_technical_indicator(tech_indicator_list)
# add industry and stock name
universe = ind_helper.add_by_basetable('ts_code', fundamental, ['industry', 'name'])

In [ ]:
universe

In [ ]:
# tmp save
save_data = universe.loc[universe.date > pd.to_datetime('2020-01-01')]
save_data.
#universe.to_csv('tmp_result.csv')

In [ ]:
# tmp load
import pandas as pd
import numpy as np
universe = pd.read_csv('tmp_result.csv',keep_date_col=True).iloc[:,1:]
universe.date = pd.to_datetime(universe.date)

In [4]:
# get fundamental data
# pick up market cap [5,30] billion
base_universe = pro.bak_daily(trade_date='20180102', 
                   fields='trade_date, ts_code, name, float_mv, total_mv, pe, turn_over, industry')
filte_stock = base_universe.loc[(base_universe.total_mv>=50) & (base_universe.total_mv<=300)]
filte_stock

,ts_code,trade_date,name,turn_over,pe,industry,float_mv,total_mv
3,600903.SH,20180102,贵州燃气,35.95,110.50,供气供热,22.62,150.81
9,000885.SZ,20180102,同力水泥,4.98,10.71,水泥,72.68,84.53
12,002372.SZ,20180102,伟星新材,1.75,27.46,其他建材,176.55,199.46
13,600507.SH,20180102,方大特钢,4.05,9.02,特种钢,185.12,185.12
14,300735.SZ,20180102,光弘科技,0.02,29.91,通信设备,14.04,56.15
...,...,...,...,...,...,...,...,...
3454,600074.SH,20180102,ST保千里,0.00,42.50,电脑设备,95.52,228.67
3455,600289.SH,20180102,ST信通,0.00,0.00,软件服务,53.42,59.57
3456,300630.SZ,20180102,普利制药,5.14,109.15,化学制药,22.02,88.08
3457,002127.SZ,20180102,南极电商,3.07,57.60,互联网,110.37,186.57


In [5]:
from helper import download_helper

In [8]:
# load total stock daily date for one year
ts_code_list = filte_stock.ts_code.values
all_stocks = download_helper.get_Daily_All(ts, ts_code_list, start_date, end_date)
print(all_stocks.shape)
all_stocks

ticker/tickers: 100%|███████████████████████| 1857/1857 [17:19<00:00,  1.79it/s]

(1319793, 17)


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,ma10,ma_v_10,ma60,ma_v_60,ma120,ma_v_120
0,600903.SH,20201231,12.8300,13.0500,12.6500,12.7000,13.0000,-0.3000,-2.3077,298018.48,380610.620,13.026,428080.779,12.1060,199368.8255,11.7891,148799.6242
1,600903.SH,20201230,12.9100,13.3500,12.9100,13.0000,13.2200,-0.2200,-1.6641,335380.15,439698.264,12.913,408187.524,12.0788,195089.4188,11.7739,147809.3177
2,600903.SH,20201229,13.5500,13.7000,13.0100,13.2200,13.7700,-0.5500,-3.9942,403904.90,539356.202,12.748,387981.164,12.0478,190053.8642,11.7531,146556.4926
3,600903.SH,20201228,13.9300,14.2100,13.3900,13.7700,13.6500,0.1200,0.8791,518263.18,713652.186,12.607,361548.627,12.0128,184151.5455,11.7313,144766.4967
4,600903.SH,20201225,13.1800,14.4800,13.0700,13.6500,13.3500,0.3000,2.2472,584433.16,807266.469,12.450,321017.163,11.9723,176815.1513,11.7012,141314.9518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,300639.SZ,20180108,27.0020,27.0020,25.2016,25.5438,27.0183,-1.4745,-5.4574,44458.13,279841.559,NaN,NaN,NaN,NaN,NaN,NaN
726,300639.SZ,20180105,27.2342,27.4705,26.6069,27.0183,26.9654,0.0529,0.1962,34242.50,227133.793,NaN,NaN,NaN,NaN,NaN,NaN
727,300639.SZ,20180104,27.3157,27.6415,26.5092,26.9654,28.3462,-1.3808,-4.8712,52177.37,346749.657,NaN,NaN,NaN,NaN,NaN,NaN
728,300639.SZ,20180103,27.6130,28.5866,27.2098,28.3462,29.1772,-0.8310,-2.8481,66187.57,450061.438,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# save data
universe = all_stocks.drop_duplicates()
universe.to_csv(start_date +'-'+ end_date + '.csv')
filte_stock.drop_duplicates().to_csv('fundamental_' + start_date +'.csv')

# Load Data by File
if we load data from saved file. 

In [2]:
# load data from csv
universe = pd.read_csv('20180101-20210101.csv').iloc[:,1:]
fundamental = pd.read_csv('fundamental_20180101.csv').iloc[:,1:]

# Process Data
1. filter average dollar volumn 120 days top 500 stocks
2. add 'date' column as datetime type, and deascanding time
3. add industry infomation and boll indicator to stock 

In [3]:
from helper.factor_helper import IndicatorHelper

ind_helper = IndicatorHelper(universe)
# pick average amount 120 days top 500
universe = ind_helper.top(500, index='trade_date', ticker_column='ts_code', value_column='ma_v_120')

# add bollinger as indicator which will be used as a custom factor later
tech_indicator_list = ['boll_ub','boll_lb']
universe = ind_helper.add_technical_indicator(tech_indicator_list)

# add industry and stock name
universe = ind_helper.add_by_basetable('ts_code', fundamental, ['industry', 'name'])

/Users/jiaruiming/PycharmProjects/altifical_for_trading/project4_Alpha_Research_and_Factor_Modeling/helper/factor_helper.py:20: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.use_column = self.columns
/Users/jiaruiming/PycharmProjects/altifical_for_trading/project4_Alpha_Research_and_Factor_Modeling/helper/factor_helper.py:63: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.stocks = stockstats.StockDataFrame.retype(self.copy())
/Users/jiaruiming/PycharmProjects/altifical_for_trading/project4_Alpha_Research_and_Factor_Modeling/helper/factor_helper.py:64: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.df = self.copy()
add fund

# Construct Factors
### Overnight Returns and Firm-Specific Investor Sentiment
the overnight return calculate by rturn=$\frac{open_t - close_{t-1}}{close_{t-1}}$
 
 paper calculate price by sum average 5 days as long factor, we just average 5 days
 
 use average 20 days of overnight return as a short factor

In [2]:
from helper.factor_helper import CloseToOpen

# cal close to open average 5day as long facor and 20day as short factor
cto = CloseToOpen(universe).calculate()
universe = cto.get_factors()
universe['close_to_open_25_sma'] = - universe['close_to_open_25_sma']

add tech indicators: 100%|██████████| 499/499 [00:09<00:00, 52.41it/s]


###  Winners and Losers in Momentum Investing
 The stock price tragectories can be expressed by $p=\mu*time + \beta*time^2$ 
 
 We convert time as linner values and get $\mu$ and $\beta$ by regression between price and constant values. 
 
 Final facotor expressed $\beta * \mu$
 
 This factor can express each stock tragectories relative convex. The $\mu$ be viewed as return direction and $\beta$ be viwed as return velocity

In [3]:
from helper.factor_helper import WinerAndLoser
wl = WinerAndLoser(universe).calculate()
universe = wl.get_factor()

 processing factors step/total 119361/119361

###  Expected Skewness and Momentum
The skewness of returns distribution and media in a period time(20 trade day) can combine to be a factor.
 
factor = $skew * median$

In [3]:
from helper.factor_helper import SkewandMomentum
sm = SkewandMomentum(universe).calculate()
universe = sm.get_factor()

skew and momentum: 100%|██████████| 499/499 [00:03<00:00, 139.03it/s]
